In [114]:
import requests
import json
response_API1 = requests.get('https://covid19.ddc.moph.go.th/api/Vaccinated/weekly-vaccinated-by-provice')
response_API2 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces')
response_API3 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-line-lists')
response_API4 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/timeline-cases-by-provinces')

#print(response_API.status_code)

In [115]:
import sqlite3

conn = sqlite3.connect('Covid.db')
cursor = conn.cursor()
#cursor.execute('PRAGMA table_info(Covid)').fetchall()

# cursor.execute('''CREATE TABLE vaccine
#                 (year int, weeknum int, province text, vaccine_total int, vaccine_1st_dose int, vaccine_2nd_dose int, vaccine_3rd_dose int, vaccine_total_acm int, vaccine_1st_dose_acm int, vaccine_2nd_dose_acm int, vaccine_3rd_dose_acm int, region text)''')

# cursor.execute('''CREATE TABLE covid
#                 (year int, weeknum int, province text, new_case int, total_case int, new_case_excludeabroad int, total_case_excludeabroad int, new_death int, total_death int, update_date text, region text)''')

# conn.commit()

In [116]:
province_north = ["เชียงราย","เชียงใหม่","น่าน","พะเยา","แพร่","แม่ฮ่องสอน","ลำปาง","ลำพูน","อุตรดิตถ์"]
province_south = ["กระบี่","ชุมพร","ตรัง","นครศรีธรรมราช","นราธิวาส","ปัตตานี","พังงา","พัทลุง","ภูเก็ต","ยะลา","ระนอง","สงขลา","สตูล","สุราษฎร์ธานี"]
province_central = ["กรุงเทพมหานคร","กำแพงเพชร","ชัยนาท","นครนายก","นครปฐม","นครสวรรค์","นนทบุรี","ปทุมธานี","พระนครศรีอยุธยา","พิจิตร","พิษณุโลก","เพชรบูรณ์","ลพบุรี","สมุทรปราการ","สมุทรสงคราม","สมุทรสาคร","สระบุรี","สิงห์บุรี","สุโขทัย","สุพรรณบุรี","อ่างทอง","อุทัยธานี"]
province_west = ["กาญจนบุรี","ตาก","ประจวบคีรีขันธ์","เพชรบุรี","ราชบุรี"]
province_east = ["จันทบุรี","ฉะเชิงเทรา","ชลบุรี","ตราด","ปราจีนบุรี","ระยอง","สระแก้ว"]
province_northeast = ["กาฬสินธุ์","ขอนแก่น","ชัยภูมิ","นครพนม","นครราชสีมา","บึงกาฬ","บุรีรัมย์","มหาสารคาม","มุกดาหาร","ยโสธร","ร้อยเอ็ด","เลย","ศรีสะเกษ","สกลนคร","สุรินทร์","หนองคาย","หนองบัวลำภู","อำนาจเจริญ","อุดรธานี","อุบลราชธานี"]

In [117]:
#ดึงข้อมูลจาก api ปี2021 - ปัจจุบัน
allcase = response_API4.text

allcase = json.loads(allcase)
for i in allcase:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    elif i["province"] in province_north:
        i["region"] = 'ภาคเหนือ'
    elif i["province"] in province_south:
        i["region"] = 'ภาคใต้'
    elif i["province"] in province_central:
        i["region"] = 'ภาคกลาง'
    elif i["province"] in province_west:
        i["region"] = 'ภาคตะวันตก'
    elif i["province"] in province_east:
        i["region"] = 'ภาคตะวันออก'
    elif i["province"] in province_northeast:
        i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    data = (i['update_date'], i['province'], i['region'], i['year'], i['weeknum'], i['new_case'], i['total_case'], i['new_case_excludeabroad'], i['total_case_excludeabroad'], i['new_death'], i['total_death'])
      
    # conn.execute("INSERT INTO covid (update_date ,province ,region ,year ,weeknum, new_case ,total_case ,new_case_excludeabroad ,total_case_excludeabroad ,new_death ,total_death ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",data)
    # conn.commit()

In [118]:
import pandas as pd
from tabulate import tabulate

vaccinated_by_provice = response_API1.text
today_cases_by_province = response_API2.text
died_cases = response_API3.text

data1 = json.loads(vaccinated_by_provice)
data2 = json.loads(today_cases_by_province)


for i in data1:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    for j in province_north:
        if i["province"] == j:
            i["region"] = 'ภาคเหนือ'
    for j in province_south:
        if i["province"] == j:
            i["region"] = 'ภาคใต้'
    for j in province_central:
        if i["province"] == j:
            i["region"] = 'ภาคกลาง'
    for j in province_west:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันตก'
    for j in province_east:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออก'
    for j in province_northeast:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    vaccine_data = tuple(i.values())
    # conn.execute("INSERT INTO vaccine VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", vaccine_data)
  


In [119]:
for i in data2:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    for j in province_north:
        if i["province"] == j:
            i["region"] = 'ภาคเหนือ'
    for j in province_south:
        if i["province"] == j:
            i["region"] = 'ภาคใต้'
    for j in province_central:
        if i["province"] == j:
            i["region"] = 'ภาคกลาง'
    for j in province_west:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันตก'
    for j in province_east:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออก'
    for j in province_northeast:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    covid_data = tuple(i.values())
    # cursor.execute("INSERT INTO covid VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", covid_data)

In [120]:
# cursor.execute('''CREATE TABLE alldata AS SELECT * FROM covid LEFT OUTER JOIN vaccine ON covid.province = vaccine.province;''')
# cursor.execute('''alter table alldata drop 'year:1' ''')
# cursor.execute('''alter table alldata drop 'weeknum:1' ''')
# cursor.execute('''alter table alldata drop 'province:1' ''')
# cursor.execute('''alter table alldata drop 'region:1' ''')

In [121]:
##เพิ่มข้อมูลเข้า table alldata
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('covid.db')

# Get a cursor object
cursor = conn.cursor()

# Execute the SELECT statement
cursor.execute("SELECT * FROM covid WHERE year != 2023")

# Fetch all the rows returned by the query
rows = cursor.fetchall()

# Insert the selected rows into the "alldata" table
for i in rows:
    data = (i +(0,0,0,0,0,0,0,0))
    # cursor.execute("INSERT INTO alldata VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?,?,?,?,?,?)", data)

# Commit the changes
# conn.commit()

# Close the cursor and connection objects
# cursor.close()
# conn.close()


In [122]:
year 2023
weeknum 9
province ทั้งประเทศ
vaccine_total 727
vaccine_1st_dose 73
vaccine_2nd_dose 153
vaccine_3rd_dose 501
vaccine_total_acm 144872431
vaccine_1st_dose_acm 57226251
vaccine_2nd_dose_acm 53719574
vaccine_3rd_dose_acm 33926606

SyntaxError: invalid syntax (2646373387.py, line 1)

In [ ]:
import urllib
url = 'https://opend.data.go.th/get-ckan/datastore_search?resource_id=0c2aa4d6-90ef-405f-acf2-20507ced4fab&limit=5&q=title:jones'  
fileobj = json.loads(requests.get(url).text)
print (fileobj)

{'message': 'No API key found in request'}


In [ ]:
output = [{'year': 2023, 'weeknum': 9, 'province': 'กระบี่', 'new_case': 0, 'total_case': 2, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 2, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สตูล', 'new_case': 0, 'total_case': 1, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 1, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลำปาง', 'new_case': 1, 'total_case': 25, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 25, 'new_death': 1, 'total_death': 9, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตราด', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ทั้งประเทศ', 'new_case': 147, 'total_case': 4263, 'new_case_excludeabroad': 147, 'total_case_excludeabroad': 4263, 'new_death': 7, 'total_death': 249, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครปฐม', 'new_case': 0, 'total_case': 42, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 42, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สิงห์บุรี', 'new_case': 0, 'total_case': 24, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 24, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุราษฎร์ธานี', 'new_case': 13, 'total_case': 61, 'new_case_excludeabroad': 13, 'total_case_excludeabroad': 61, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พัทลุง', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุทัยธานี', 'new_case': 0, 'total_case': 39, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 39, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ยะลา', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปัตตานี', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'หนองคาย', 'new_case': 1, 'total_case': 20, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 20, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปราจีนบุรี', 'new_case': 5, 'total_case': 42, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 42, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พังงา', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ยโสธร', 'new_case': 0, 'total_case': 20, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 20, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'หนองบัวลำภู', 'new_case': 0, 'total_case': 31, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 31, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ราชบุรี', 'new_case': 0, 'total_case': 40, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 40, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครสวรรค์', 'new_case': 0, 'total_case': 75, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 75, 'new_death': 0, 'total_death': 7, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลำพูน', 'new_case': 0, 'total_case': 59, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 59, 'new_death': 0, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กาญจนบุรี', 'new_case': 3, 'total_case': 19, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ระนอง', 'new_case': 0, 'total_case': 1, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 1, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เชียงราย', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'จันทบุรี', 'new_case': 2, 'total_case': 55, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 55, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พิษณุโลก', 'new_case': 0, 'total_case': 60, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 60, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นราธิวาส', 'new_case': 0, 'total_case': 3, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 3, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กรุงเทพมหานคร', 'new_case': 40, 'total_case': 728, 'new_case_excludeabroad': 40, 'total_case_excludeabroad': 728, 'new_death': 4, 'total_death': 44, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นนทบุรี', 'new_case': 4, 'total_case': 124, 'new_case_excludeabroad': 4, 'total_case_excludeabroad': 124, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ประจวบคีรีขันธ์', 'new_case': 1, 'total_case': 32, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'บึงกาฬ', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชลบุรี', 'new_case': 10, 'total_case': 205, 'new_case_excludeabroad': 10, 'total_case_excludeabroad': 205, 'new_death': 0, 'total_death': 9, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อ่างทอง', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'แพร่', 'new_case': 0, 'total_case': 15, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 15, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ศรีสะเกษ', 'new_case': 0, 'total_case': 38, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 38, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เพชรบูรณ์', 'new_case': 1, 'total_case': 17, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 17, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุดรธานี', 'new_case': 0, 'total_case': 44, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 44, 'new_death': 0, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กาฬสินธุ์', 'new_case': 5, 'total_case': 73, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 73, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กำแพงเพชร', 'new_case': 0, 'total_case': 25, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 25, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ระยอง', 'new_case': 1, 'total_case': 45, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 45, 'new_death': 0, 'total_death': 7, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครศรีธรรมราช', 'new_case': 0, 'total_case': 32, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชุมพร', 'new_case': 1, 'total_case': 124, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 124, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เลย', 'new_case': 0, 'total_case': 82, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 82, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลพบุรี', 'new_case': 2, 'total_case': 47, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 47, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรสาคร', 'new_case': 0, 'total_case': 27, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 27, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เชียงใหม่', 'new_case': 0, 'total_case': 18, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 18, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ขอนแก่น', 'new_case': 2, 'total_case': 112, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 112, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'มุกดาหาร', 'new_case': 0, 'total_case': 10, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 10, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พิจิตร', 'new_case': 0, 'total_case': 33, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 33, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชัยนาท', 'new_case': 0, 'total_case': 49, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 49, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เพชรบุรี', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุบลราชธานี', 'new_case': 9, 'total_case': 200, 'new_case_excludeabroad': 9, 'total_case_excludeabroad': 200, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชัยภูมิ', 'new_case': 0, 'total_case': 35, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 35, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ร้อยเอ็ด', 'new_case': 0, 'total_case': 65, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 65, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครราชสีมา', 'new_case': 3, 'total_case': 108, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 108, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตาก', 'new_case': 2, 'total_case': 32, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุรินทร์', 'new_case': 3, 'total_case': 189, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 189, 'new_death': 0, 'total_death': 12, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อำนาจเจริญ', 'new_case': 0, 'total_case': 7, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 7, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุโขทัย', 'new_case': 0, 'total_case': 28, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 28, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สระบุรี', 'new_case': 1, 'total_case': 24, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 24, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครพนม', 'new_case': 7, 'total_case': 47, 'new_case_excludeabroad': 7, 'total_case_excludeabroad': 47, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สกลนคร', 'new_case': 2, 'total_case': 48, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 48, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พะเยา', 'new_case': 0, 'total_case': 19, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สระแก้ว', 'new_case': 3, 'total_case': 54, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 54, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรปราการ', 'new_case': 0, 'total_case': 104, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 104, 'new_death': 0, 'total_death': 10, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พระนครศรีอยุธยา', 'new_case': 1, 'total_case': 28, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 28, 'new_death': 1, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุตรดิตถ์', 'new_case': 2, 'total_case': 25, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 25, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครนายก', 'new_case': 2, 'total_case': 43, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 43, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'น่าน', 'new_case': 3, 'total_case': 109, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 109, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ไม่ระบุ', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ฉะเชิงเทรา', 'new_case': 0, 'total_case': 40, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 40, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุพรรณบุรี', 'new_case': 4, 'total_case': 75, 'new_case_excludeabroad': 4, 'total_case_excludeabroad': 75, 'new_death': 1, 'total_death': 10, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'บุรีรัมย์', 'new_case': 5, 'total_case': 174, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 174, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตรัง', 'new_case': 0, 'total_case': 18, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 18, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'แม่ฮ่องสอน', 'new_case': 0, 'total_case': 19, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'มหาสารคาม', 'new_case': 1, 'total_case': 28, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 28, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรสงคราม', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สงขลา', 'new_case': 1, 'total_case': 39, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 39, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ภูเก็ต', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปทุมธานี', 'new_case': 6, 'total_case': 75, 'new_case_excludeabroad': 6, 'total_case_excludeabroad': 75, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}]


In [ ]:
for i in 
cursor.execute("INSERT INTO  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", vaccine_data)


SyntaxError: invalid syntax (3091824406.py, line 1)

In [ ]:
import json
import sqlite3

# Open the JSON file and load its contents
with open('Thailandprovinces.json', 'r') as f:
    province = json.load(f)
# Extract the new value for the "province" column
# Thailandprovinces

# # Connect to the SQLite database
conn = sqlite3.connect('covid.db')

# # Get a cursor object
cursor = conn.cursor()

# # Execute the UPDATE statement
# for i in data.keys():
#     new_province = data.get(i)
#     #print(i,new_province)
#     conn.execute("UPDATE alldata SET province = ? WHERE province = ?", (new_province, i))
#     conn.commit()
    
# conn.execute("UPDATE alldata set province = ? WHERE province = ?", ('None', 'ไม่ระบุ'))
# conn.commit()
# conn.execute("UPDATE alldata set province = ? WHERE province = ?", ('All', 'ทั้งประเทศ'))
# conn.commit()
# # Commit the changes



In [ ]:
region = {"ทั้งประเทศ":"All","ไม่ระบุ":'None','ภาคเหนือ':'North','ภาคใต้':'South','ภาคกลาง':'Central','ภาคตะวันตก':'West','ภาคตะวันออก':'East','ภาคตะวันออกเฉียงเหนือ':'NorthEast'}
for i in region.keys():
    new_region = region.get(i)
    # conn.execute("UPDATE alldata_province_eng SET region = ? WHERE region = ?", (i, new_region))
    # conn.commit()

In [131]:
for i in province.keys():
    new_province = province.get(i)
#     conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", (i, new_province))
#     conn.commit()

# conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", ("ทั้งประเทศ", "All"))
# conn.commit()
# conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", ("ไม่ระบุ", "None"))
# conn.commit()

